In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import GridSearchCV
X_Train=pd.read_csv('2014-XGboost-Train.csv',engine = 'python')
y_Test = pd.read_csv('2014-XGboost-TEST.csv',engine = 'python')
#xgbt = xgb.XGBRegressor(tree_method= 'gpu_hist',gpu_id = -1,max_bin =16)
#xgbt = xgb.XGBRegressor()
xgbt = xgb.XGBRegressor()
#param_dist = {'n_estimators':list(range(150,450,90)),
 #       'max_depth':list(range(3,8,2)),
  #      'learning_rate':list(np.linspace(0.01,0.3,2)),
   #     'min_child_weight':list(range(5,7,1)),
    #      }
param_dist = {'n_estimators':list(range(170,450,30)),
        'max_depth':list(range(2,9,2)),
        'learning_rate':list(np.linspace(0.01,0.3,2,8)),
        'min_child_weight':list(range(1,7,1)),
          }

y_Test= y_Test[:7*24]
target = []
MIC_order = pd.read_csv('MIC3.csv')
for i in range(0,37,1):
    target.append(MIC_order['区域顺序'][i])
feature1 = ['month', 'week', 'hour', 'weather', 'temperature']
MIC_all2 = pd.read_csv('目标之间的MIC系数矩阵.csv',engine = 'python')
MIC_all2 = MIC_all2[['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']]
MIC_all2.index = ['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']
def fun(da,a):
    for i in da.columns:
        for j in da.index:
            if da[i][j] > a:
                da[i][j]=1
fun(MIC_all2,0.549796)
MIC = MIC_all2
target = []
MIC_order = pd.read_csv('MIC3.csv')
for i in range(0,37,1):
    target.append(MIC_order['区域顺序'][i])
feature1 = ['month', 'week', 'hour', 'weather', 'temperature']
DARC = pd.DataFrame()
for i in range(0,37,1):
    sorted_idx = []
    feature = []
    sorted_idx = np.argsort(MIC[target[i]][MIC[target[i]].isin([1])])
    sorted_idx=list(sorted_idx.index)
    sorted_idx.reverse()
    feature2 = [val for val in sorted_idx if val in list(DARC.columns)]
    #feature2 =list(DARC.columns)
    feature = feature1+feature2
    g1 =  X_Train[feature].values
    g1_test = X_Train[target[i]].values
    X_test = y_Test[feature].values
    Y_test = y_Test[target[i]].values
    print(target[i],'模型的特征',feature)
    #print(g1,g1_test,X_test,Y_test )
    xgbt_T = GridSearchCV(xgbt,param_dist,cv = 3,scoring = 'r2',n_jobs = -1)
    xgbt_T.fit(g1,g1_test)
    print(xgbt_T.best_score_)
    #xgbt = XGBRegressor(max_depth=5, learning_rate=0.15, n_estimators=200, min_child_weight=1,)
    #xgbt_T.best_estimator_.fit(g1,g1_test)
    y_pred = xgbt_T.best_estimator_.predict(X_test)
    MSE = mean_squared_error(Y_test, y_pred)
    
    print(target[i],'测试集均方根误差：',MSE **0.5) 
    y_pred = pd.Series(y_pred,name = target[i])
    DARC = pd.concat([DARC,y_pred], axis=1)
DARC=DARC[['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']]
DARC.to_csv('MICDARC0708_1.csv')

grid23 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature']
[23:22:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9294588015130499
grid23 测试集均方根误差： 57.90922601162647
grid1 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23']
[23:23:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9335103146453734
grid1 测试集均方根误差： 25.335894322496184
grid2 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23', 'grid1']
[23:24:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9195000088364451
grid2 测试集均方根误差： 15.517805876596709
grid26 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23', 'grid2', 'grid1']
[23:26:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.8828687687457695
grid26 测试集均方根误差： 43.68100096223239
grid3

0.9196265787132067
grid34 测试集均方根误差： 13.043527327329835
grid32 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid37', 'grid35', 'grid34', 'grid33', 'grid31', 'grid30', 'grid28', 'grid27', 'grid26', 'grid25', 'grid24', 'grid23', 'grid22', 'grid20', 'grid19', 'grid17', 'grid15', 'grid13', 'grid3', 'grid2', 'grid1']
[00:34:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9577506826110308
grid32 测试集均方根误差： 21.108066391890958
grid18 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid37', 'grid35', 'grid31', 'grid27', 'grid26', 'grid23', 'grid22', 'grid20', 'grid19', 'grid17', 'grid16', 'grid15', 'grid14', 'grid13', 'grid10', 'grid3', 'grid2', 'grid1']
[00:38:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9445319472660088
grid18 测试集均方根误差： 76.29982051281333
grid11 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid31', 'grid28', 'grid

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import GridSearchCV
X_Train=pd.read_csv('2014-XGboost-Train.csv',engine = 'python')
y_Test = pd.read_csv('2014-XGboost-TEST.csv',engine = 'python')
#xgbt = xgb.XGBRegressor(tree_method= 'gpu_hist',gpu_id = -1,max_bin =16)
#xgbt = xgb.XGBRegressor()
xgbt = xgb.XGBRegressor()
#param_dist = {'n_estimators':list(range(150,450,90)),
 #       'max_depth':list(range(3,8,2)),
  #      'learning_rate':list(np.linspace(0.01,0.3,2)),
   #     'min_child_weight':list(range(5,7,1)),
    #      }
param_dist = {'n_estimators':list(range(170,450,30)),
        'max_depth':list(range(2,9,2)),
        'learning_rate':list(np.linspace(0.01,0.3,8)),
        'min_child_weight':list(range(2,8,1)),
          }

y_Test= y_Test[:7*24]
target = []
MIC_order = pd.read_csv('MIC3.csv')
for i in range(0,37,1):
    target.append(MIC_order['区域顺序'][i])
feature1 = ['month', 'week', 'hour', 'weather', 'temperature']
MIC_all2 = pd.read_csv('目标之间的MIC系数矩阵.csv',engine = 'python')
MIC_all2 = MIC_all2[['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']]
MIC_all2.index = ['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']
def fun(da,a):
    for i in da.columns:
        for j in da.index:
            if da[i][j] > a:
                da[i][j]=1
fun(MIC_all2,0.549796)
MIC = MIC_all2
target = []
MIC_order = pd.read_csv('MIC3.csv')
for i in range(0,37,1):
    target.append(MIC_order['区域顺序'][i])
feature1 = ['month', 'week', 'hour', 'weather', 'temperature']
DARC = pd.DataFrame()
for i in range(0,37,1):
    sorted_idx = []
    feature = []
    sorted_idx = np.argsort(MIC[target[i]][MIC[target[i]].isin([1])])
    sorted_idx=list(sorted_idx.index)
    sorted_idx.reverse()
    feature2 = [val for val in sorted_idx if val in list(DARC.columns)]
    #feature2 =list(DARC.columns)
    feature = feature1+feature2
    g1 =  X_Train[feature].values
    g1_test = X_Train[target[i]].values
    X_test = y_Test[feature].values
    Y_test = y_Test[target[i]].values
    print(target[i],'模型的特征',feature)
    #print(g1,g1_test,X_test,Y_test )
    xgbt_T = GridSearchCV(xgbt,param_dist,cv = 3,scoring = 'r2',n_jobs = -1)
    xgbt_T.fit(g1,g1_test)
    print(xgbt_T.best_score_)
    #xgbt = XGBRegressor(max_depth=5, learning_rate=0.15, n_estimators=200, min_child_weight=1,)
    #xgbt_T.best_estimator_.fit(g1,g1_test)
    y_pred = xgbt_T.best_estimator_.predict(X_test)
    MSE = mean_squared_error(Y_test, y_pred)
    
    print(target[i],'测试集均方根误差：',MSE **0.5) 
    y_pred = pd.Series(y_pred,name = target[i])
    DARC = pd.concat([DARC,y_pred], axis=1)
DARC=DARC[['grid1', 'grid2', 'grid3', 'grid4', 'grid5', 'grid6',
       'grid7', 'grid8', 'grid9', 'grid10', 'grid11', 'grid12', 'grid13',
       'grid14', 'grid15', 'grid16', 'grid17', 'grid18', 'grid19', 'grid20',
       'grid21', 'grid22', 'grid23', 'grid24', 'grid25', 'grid26', 'grid27',
       'grid28', 'grid29', 'grid30', 'grid31', 'grid32', 'grid33', 'grid34',
       'grid35', 'grid36', 'grid37']]
DARC.to_csv('MICDARC0708_2.csv')

grid23 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature']
[08:38:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9309053633529573
grid23 测试集均方根误差： 57.74806323851143
grid1 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23']
[08:48:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9351198018800533
grid1 测试集均方根误差： 24.99961898096735
grid2 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23', 'grid1']
[08:59:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9204495494409237
grid2 测试集均方根误差： 15.444142031426395
grid26 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid23', 'grid2', 'grid1']
[09:10:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.8830169355913597
grid26 测试集均方根误差： 48.13576557026627
grid31

0.922627649197926
grid34 测试集均方根误差： 13.55213637314884
grid32 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid37', 'grid35', 'grid34', 'grid33', 'grid31', 'grid30', 'grid28', 'grid27', 'grid26', 'grid25', 'grid24', 'grid23', 'grid22', 'grid20', 'grid19', 'grid17', 'grid15', 'grid13', 'grid3', 'grid2', 'grid1']
[17:43:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9584416328243539
grid32 测试集均方根误差： 23.97699098024493
grid18 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid37', 'grid35', 'grid31', 'grid27', 'grid26', 'grid23', 'grid22', 'grid20', 'grid19', 'grid17', 'grid16', 'grid15', 'grid14', 'grid13', 'grid10', 'grid3', 'grid2', 'grid1']
[18:12:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9492739220450273
grid18 测试集均方根误差： 69.49405100825965
grid11 模型的特征 ['month', 'week', 'hour', 'weather', 'temperature', 'grid31', 'grid28', 'grid26'